In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec

In [0]:
downloaded = drive.CreateFile({'id':'1UDiOho3fJT_VRssVjn9ZI9SrBI34Fkh9'}) ## Load Secondary_Source.csv in the edge_list_creation folder
downloaded.GetContentFile('secondary_source.csv')  
file_dictionary = pd.read_csv('secondary_source.csv')
del file_dictionary['Unnamed: 0']

In [0]:
file_dictionary

,Title,Content
0,Llop_2010_AuOr_28_Rec._OBO_160_5.txt,"Aula Orientalis 28, 2010, 139- 155 (ISSN: 0212..."
1,pardee2001b.txt,Alphabet theofDevelopment and Origin The\nIsra...
2,Hecker_1978b_OrNS47_tib'imma.txt,404\ntibimma atalkim\nAssyrerinnen im karumzei...
3,pardee1979-80a.txt,"ey\nIgnatios Ya‘qub 1918; II{, Liban Hissiyah,..."
4,"Lassen, Agnete - Glyptic Encounters Appendicie...",APPENDIX 3 APPENDIX 3: IDENTIFIED SEAL OWNERS\...
...,...,...
1341,Review of Brak.txt,"Excavationsat Tell Brak, Vol. 1: The Mitanni a..."
1342,pru3pt1002.txt,"LE PALAIS DUGARIT 303\nadopte pourfils, en pré..."
1343,Ivanov_PIES2006.txt,Proceedings of the 19th\nAnnual UCLA Indo-Euro...
1344,McCreedy_2002_JNES63_Rev_Schloen_PHM.txt,232 Journal of Near Eastern Studies Vol. 63 No...


In [0]:
file = file_dictionary.values

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
### Get the stopwords from different languages from nltk package
french_stopwords = stopwords.words('french')
german_stopwords = stopwords.words('german')
turkish_stopwords = stopwords.words('turkish')

In [0]:
## This is a function designed to remove all the stopwords in our text
def preprocess(text): # one little problem, gensim.utils.simple_preprocess gets rid off "I" automatically. 
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
          if token not in french_stopwords:
            if token not in german_stopwords:
              if token not in turkish_stopwords:
                result.append(token)
    return result

In [0]:
file_dictionary = {}
for i in file:
  file_dictionary[i[0]] = i[1]

In [0]:
for key in file_dictionary:
  #print("working on file "+ key)
  file_dictionary[key] = preprocess(file_dictionary[key])

In [0]:
##this function tag all processed files so it is the form for Doc2Vec model 
def tag_doc(processed, i):
  return gensim.models.doc2vec.TaggedDocument(processed, [str(i)])

In [0]:
temp_a = [(k,v) for k,v in file_dictionary.items()]

In [0]:
temp = [i[1] for i in temp_a]

In [0]:
processed_files = []
for i in range(len(temp)):
  processed_files.append(tag_doc(temp[i], i))

In [0]:
test_files = []
for i in range(len(temp)):
  test_files.append(temp[i])

In [0]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=1, epochs=40)

In [0]:
model.build_vocab(processed_files)

In [0]:
model.train(processed_files, total_examples=model.corpus_count, epochs=model.epochs)

In [0]:
threshold = 0.9 #
Source = []
Target = []
Weight = []
number_of_doc = 500 ##change it 1346 for all

In [0]:
for i in range(number_of_doc):
  for k in range(i+1, number_of_doc):
    print("working on " + str(i) + "comparing "str(k))
    similar = model.n_similarity(test_files[i],test_files[k])
    if similar > threshold:
      Source.append(file[i][0])
      Target.append(file[k][0])
      Weight.append(similar) 

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Similarity':Weight})

In [0]:
result

Important Note: To check each possible edge, there are around 500000 checks. Each check takes a second, it will cost us around 5 days to check all the edges.

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
result.to_csv('Doc2Vec_edge_list.csv', index=False)
!cp Doc2Vec_edge_list.csv drive/My\ Drive/AWCA/Collab_notebooks/edge_list_creation/